In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deep-learning-spring-2025-project-1/cifar_test_nolabel.pkl
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/data_batch_1
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/data_batch_2
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/batches.meta
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/test_batch
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/data_batch_3
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/data_batch_5
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/data_batch_4
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/readme.html


In [2]:
!git clone https://github.com/timothycao/cifar10_classifier.git
%cd cifar10_classifier

Cloning into 'cifar10_classifier'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 90 (delta 51), reused 65 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 23.59 KiB | 4.72 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/kaggle/working/cifar10_classifier


In [3]:
import torch.optim as optim
import torchvision.transforms as transforms
from model import create_model
from train import main as train
from inference import main as inference


# Model parameters
MODEL_NAME = 'ResNetCustom'
BLOCK_TYPE = 'bottleneck'
NUM_BLOCKS_PER_LAYER = [3, 4, 6, 3]
NUM_CHANNELS_PER_LAYER = [29, 58, 116, 232]
KERNEL_SIZE_PER_LAYER = [3, 3, 3, 3]
SKIP_KERNEL_SIZE_PER_LAYER = [1, 1, 1, 1]
EXPANSION = 4
POOL_SIZE = 1


# Training parameters
EPOCHS = 200
TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 100
AUGMENTATIONS = [
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.RandAugment(num_ops=2, magnitude=12),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.RandomGrayscale(p=0.2)
]
SAVE_MODE = 'best'  # Options: 'best', 'every'
SAVE_EVERY_N = 1

# Initialize model
try:
    model = create_model(
        name=MODEL_NAME,
        block_type=BLOCK_TYPE,
        blocks_per_layer=NUM_BLOCKS_PER_LAYER,
        channels_per_layer=NUM_CHANNELS_PER_LAYER,
        kernels_per_layer=KERNEL_SIZE_PER_LAYER,
        skip_kernels_per_layer=SKIP_KERNEL_SIZE_PER_LAYER,
        expansion=EXPANSION,
        pool_size=POOL_SIZE
    )
    print('Total model parameters:', sum(p.numel() for p in model.parameters() if p.requires_grad))
except AssertionError as e:
    print(f'Failed to create model: {e}')
    exit(1)

Total model parameters: 4848027


In [ ]:
# Train model
try:
    # Define optimizer
    OPTIMIZER = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=8e-4)
    
    # Define scheduler
    SCHEDULER = optim.lr_scheduler.CosineAnnealingLR(OPTIMIZER, T_max=EPOCHS)
    
    # If no scheduler, set to None
    SCHEDULER = SCHEDULER if 'SCHEDULER' in locals() and SCHEDULER is not None else None

    # train(model, EPOCHS)
    train(model, EPOCHS, train_batch_size=TRAIN_BATCH_SIZE, test_batch_size=TEST_BATCH_SIZE, augmentations=AUGMENTATIONS,
          optimizer=OPTIMIZER, scheduler=SCHEDULER, save=SAVE_MODE, every_n=SAVE_EVERY_N)
except (ValueError, TypeError) as e:
    print(f'Training failed: {e}')
    exit(1)

Total model parameters: 4848027
Loading data...
Preprocessing pipeline:
 Compose(
    RandomHorizontalFlip(p=0.5)
    RandomCrop(size=(32, 32), padding=4)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.1, 0.1))
    RandomAffine(degrees=[-10.0, 10.0], translate=(0.1, 0.1), scale=(0.8, 1.2))
    RandAugment(num_ops=2, magnitude=12, num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
    RandomGrayscale(p=0.2)
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
)
Files already downloaded and verified
Files already downloaded and verified
Initializing model...
Training model...

Epoch: 1/200      LR: 0.01000
TRAIN: Loss: 2.13 Acc: 21.22%
TEST:  Loss: 1.78 Acc: 34.99%

Epoch: 2/200      LR: 0.01000
TRAIN: Loss: 1.86 Acc: 31.62%
TEST:  Loss: 1.67 Acc: 39.66%

Epoch: 3/200      LR: 0.01000
TRAIN: Loss: 1.72 Acc: 37.62%
TEST:  Loss: 1.45

In [ ]:
# Run inference
try:
    inference('ResNetCustom_epoch182_acc94.pth')
except FileNotFoundError as e:
    print(f'Inference failed: {e}')

Loading data...
Loading model...
Running inference...
Predictions saved as ResNetCustom_epoch182_acc94.csv
